In [1]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv, find_dotenv
import congress
import agent

genai.__version__

'1.7.0'

Uncomment this if running in kaggle and make sureyou have secrets for the GOOGLE_API_KEY (available [here](https://aistudio.google.com/app/apikey)) and CONGRESS_API KEY (available [here](https://api.congress.gov/sign-up/)).
If you are running locally make sure you have those keys set in your .env

In [2]:
#from kaggle_secrets import UserSecretsClient
#os.environ["GOOGLE_API_KEY"] = UserSecretsClient().get_secret("GOOGLE_API_KEY")
#os.environ["CONGRESS_API_KEY"] = UserSecretsClient().get_secret("CONGRESS_API_KEY")

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
a = agent.CongressAgent()

In [ ]:
a.run()

--> can you find some bills that might impact tariff rates?


-- calling query_bill_summaries(**{'query': 'tariff rates', 'n': 5})
Here are some bills related to tariff rates:
* **H.R. 1804**: Honest Runway Labeling Act - This bill requires the President to designate certain primary airports near the border as ports of entry and terminate the user fee requirement for customs services at these airports.
* **H.R. 6454**: Chemical Tax Repeal Act - This bill repeals the excise taxes on certain taxable chemicals and imported substances.
* **H.R. 4695**: Unfair Tax Prevention Act - This bill imposes reciprocal taxes on foreign jurisdictions that attempt to impose certain surtaxes on U.S. business operations.
* **H.R. 4148**: Import Security and Fairness Act - This bill excludes imported articles from nonmarket economy countries or countries on the priority watch list from receiving de minimis treatment.
* **H.R. 4217**: Secure E-Waste Export and Recycling Act - This bill prohibits the export or reexport of electronic waste, such as computers, televisio

--> tell me some about that first one


-- calling call_endpoint(**{'endpoint': '/bill/118/hr/1804'})
The Honest Runway Labeling Act requires the President to designate certain primary airports near the border as ports of entry and terminate the user fee requirement for customs services at these airports. It was introduced on March 27, 2023, and referred to the House Committee on Ways and Means. The bill's sponsor is Rep. Gonzalez, Vicente [D-TX-34].


--> does it have any cosponsors?


...

-- calling call_endpoint(**{'endpoint': 'https://api.congress.gov/v3/bill/118/hr/3796?format=json'})
It looks like H.R. 3796 would extend taxes funding the Airport and Airway Trust Fund and require the designation of certain airports as ports of entry.



--> who is its sponsor?


The sponsor of HR 3796 is Rep. Smith, Jason [R-MO-8].



--> tell me about him


-- calling call_endpoint(**{'endpoint': 'https://api.congress.gov/v3/member/S001195?format=json'})
That's Representative Jason Smith from Missouri, a Republican serving in the House of Representatives since 2013.

